# Board games are awesome!
And we're going to find out the most awesome of qualities 

In [1]:
import pandas as pd
import numpy as np

import acquire

## Acquire

In [6]:
# games = acquire.scrape_search(1)
# for count in range (2,11):
#     games.extend(acquire.scrape_search(count))
    
for count in range (8,21):
    games.extend(acquire.scrape_search(count))

203427
the following rank has just been pulled 701
165838


TypeError: 'NoneType' object is not subscriptable

In [3]:
games[-1]

{'name_clean': 'Netrunner',
 'rank': '700',
 'geek_rating': '\n\t\t\t6.690\t\t',
 'avg_rating': '\n\t\t\t7.46\t\t',
 'votes': '\n\t\t\t2258\t\t',
 'name': 'Netrunner',
 'year': '1996',
 'min_players': '2',
 'max_players': '2',
 'min_time': '45',
 'max_time': '45',
 'designer': 'Richard Garfield',
 'family': ['Card Game', 'Collectible Components', 'Science Fiction'],
 'mechanic': ['Action Points'],
 'publisher': ['Wizards of the Coast'],
 'description': "Netrunner is a cyberpunk-themed collectible card game.  The two players are the Corp (who wins by completing Agendas) and the Runner (who wins by infiltrating the Corp and liberating Agendas).&#10;&#10;Each side has a limited number of both actions and bits (money) to spend per turn; bits are gone once used, and actions must be spent to gain more.&#10;An important thing when evaluating the power of cards is to see the Bit per action ratio.&#10;&#10;One unique feature of this game is that the Runner physically moves on the &quot;board&qu

In [4]:
pd.DataFrame(games).set_index('rank').to_csv("games700.csv")
df = pd.DataFrame(games).set_index('rank')

In [20]:
import re

## Prepare

In [21]:
df

,name_clean,geek_rating,avg_rating,votes,name,year,min_players,max_players,min_time,max_time,designer,family,mechanic,publisher,description
rank,,,,,,,,,,,,,,,
1,Gloomhaven,\n\t\t\t8.587\t\t,\n\t\t\t8.86\t\t,\n\t\t\t30159\t\t,Gloomhaven,2017,1,4,60,120,Isaac Childres,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Campaign / Battle Card Driven, Cooperative Ga...","[Cephalofair Games, Albi, Asmodee, Feuerland S...",Gloomhaven is a game of Euro-inspired tactica...
2,Pandemic Legacy: Season 1,\n\t\t\t8.473\t\t,\n\t\t\t8.63\t\t,\n\t\t\t34114\t\t,Pandemic Legacy: Season 1,2015,2,4,60,60,Rob Daviau,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...","[Z-Man Games, Inc., Asterion Press, Devir, Fil...",Pandemic Legacy is a co-operative campaign gam...
3,Terraforming Mars,\n\t\t\t8.261\t\t,\n\t\t\t8.42\t\t,\n\t\t\t46739\t\t,Terraforming Mars,2016,1,5,120,120,Jacob Fryxelius,"[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem...","[FryxGames, Arclight, Fantasmagoria, Ghenos Ga...","In the 2400s, mankind begins to terraform the ..."
4,Through the Ages: A New Story of Civilization,\n\t\t\t8.239\t\t,\n\t\t\t8.50\t\t,\n\t\t\t17888\t\t,Through the Ages: A New Story of Civilization,2015,2,4,120,120,Vlaada Chvátil,"[Card Game, Civilization, Economic]","[Action Points, Auction/Bidding, Auction: Dutc...","[Czech Games Edition, Cranio Creations, Devir,...",Through the Ages: A New Story of Civilization ...
5,Brass: Birmingham,\n\t\t\t8.187\t\t,\n\t\t\t8.62\t\t,\n\t\t\t9358\t\t,Brass: Birmingham,2018,2,4,60,120,Gavan Brown,"[Economic, Industry / Manufacturing, Transport...","[Hand Management, Income, Loans, Market, Netwo...","[Roxley, BoardM Factory, Conclave Editora, Cro...",Brass: Birmingham is an economic strategy game...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,BattleLore,\n\t\t\t7.070\t\t,\n\t\t\t7.37\t\t,\n\t\t\t8973\t\t,BattleLore,2006,2,2,60,60,Richard Borg,"[Fantasy, Medieval, Miniatures, Wargame]","[Campaign / Battle Card Driven, Command Cards,...","[Days of Wonder, Fantasy Flight Games]",This game is based upon Richard Borg's Command...
297,Western Legends,\n\t\t\t7.068\t\t,\n\t\t\t7.70\t\t,\n\t\t\t3928\t\t,Western Legends,2018,2,6,60,90,Hervé Lemaître,[American West],"[Action Points, Area Movement, Betting and Blu...","[Kolossal Games, Board Game Box, Funiverse, Gé...",Historic characters from the American Wild Wes...
298,Firefly: The Game,\n\t\t\t7.068\t\t,\n\t\t\t7.39\t\t,\n\t\t\t9615\t\t,Firefly: The Game,2013,1,4,120,240,Aaron Dill,"[Movies / TV / Radio theme, Science Fiction, S...","[Area Movement, Card Drafting, Dice Rolling, P...","[Gale Force Nine, LLC, Battlefront Miniatures ...","Players begin with a ship, and travel from pla..."


In [24]:
df.geek_rating = df.geek_rating.str.extract(r'(\d+.\d+)')
df.avg_rating = df.avg_rating.str.extract(r'(\d+.\d+)')
df.votes = df.votes.str.extract(r'(\d+.\d+)')

In [26]:
df [df.name_clean != df.name]

,name_clean,geek_rating,avg_rating,votes,name,year,min_players,max_players,min_time,max_time,designer,family,mechanic,publisher,description
rank,,,,,,,,,,,,,,,
